In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import numba as nb
from numbalsoda import lsoda, address_as_void_pointer

from src.lr import gernerate_full_lr_with_oxalate
from src.loss import cr_log, args_dtype_cr, create_jit_cr
from src.chemkinetics import ChemKinetics, find_sparse_model
from src.load_data import generate_d
from src.tools import get_conservation, generate_chem_formula, chem_equation_with_oxalate
from src.report import report

In [2]:
# set the number of threads to be used by numba
#print(nb.config.NUMBA_DEFAULT_NUM_THREADS)
nb.config.NUMBA_NUM_THREADS = 8
print(nb.config.NUMBA_NUM_THREADS)

# create the function (rate equation in this study) to be called by numbalsoda
cr_cfunc = create_jit_cr(cr_log, args_dtype_cr)
funcptr_cr = cr_cfunc.address

8


In [3]:
conservation = [[1, 1, 1, 0, 0, 1, 1, 0,], # number of Mn atoms
                [0, 0, 0, 2, 1, 0, 0, 2,], # number of C atoms
                [7, 3, 2,-2, 0, 6, 4,-1,], # charge state
               ] # Mn(7+), Mn(3+), Mn(2+), C2O4(2-), CO2, Mn(6+), Mn(4+), C2O4(-)

chemformula = generate_chem_formula(conservation)
l,r = gernerate_full_lr_with_oxalate(conservation)
print(chem_equation_with_oxalate(l, r, k=None, chemformula=chemformula, conservation=conservation))

( 1) Mn+7 + Mn+3     -> Mn+6 + Mn+4     k = 0.00
( 2) Mn+7 + Mn+2     -> Mn+3 + Mn+6     k = 0.00
( 3) Mn+7 + C2O4-2   -> Mn+6 + C2O4-1   k = 0.00
( 4) 2 Mn+3          -> Mn+2 + Mn+4     k = 0.00
( 5) Mn+3 + C2O4-2   -> Mn+2 + C2O4-1   k = 0.00
( 6) Mn+3 + Mn+6     -> Mn+7 + Mn+2     k = 0.00
( 7) Mn+3 + C2O4-1   -> C2O4-2 + Mn+4   k = 0.00
( 8) Mn+2 + Mn+6     -> 2 Mn+4          k = 0.00
( 9) Mn+2 + Mn+4     -> 2 Mn+3          k = 0.00
(10) Mn+2 + C2O4-1   -> Mn+3 + C2O4-2   k = 0.00
(11) C2O4-2 + Mn+4   -> Mn+3 + C2O4-1   k = 0.00
(12) Mn+6 + Mn+4     -> Mn+7 + Mn+3     k = 0.00
(13) Mn+6 + C2O4-1   -> Mn+7 + C2O4-2   k = 0.00
(14) 2 Mn+4          -> Mn+2 + Mn+6     k = 0.00
(15) Mn+7 + C2O4-1   -> 2 CO2 + Mn+6    k = 0.00
(16) Mn+3 + C2O4-1   -> Mn+2 + 2 CO2    k = 0.00
(17) C2O4-2 + Mn+6   -> 2 CO2 + Mn+4    k = 0.00
(18) C2O4-2 + Mn+4   -> Mn+2 + 2 CO2    k = 0.00
(19) Mn+4 + C2O4-1   -> Mn+3 + 2 CO2    k = 0.00
(20) 2 C2O4-1        -> C2O4-2 + 2 CO2  k = 0.00



In [ ]:
# find full models (lam = 0) and models for any given lam
savefolder = "example/init"
savename = "0000"
lam = 0.
target = 0. 
# Once the MRSE value decreases to less than the target value, the tentative best result is injected as an external solution to CMA-ES.
# This does not work when target value = 0.

d, d_test = generate_d()
ck = ChemKinetics(funcptr_cr, d, d_test=d_test, l=l, r=r, conservation=conservation, chemformula=chemformula)
ck.optimize(lam = lam, target=target)
ck.save(savefolder, savename)

In [ ]:
# Output omitted for brevity

In [ ]:
# find simplified models
loadpath = "example/init"
fname = "0000"

d, d_test = generate_d()
init_model = ChemKinetics(funcptr_cr, d, d_test=d_test)
init_model.load(loadpath, fname)
savepath = "example/sparse"
find_sparse_model(init_model, savepath)

In [ ]:
# Output omitted for brevity

In [6]:
# load a kinetic model and create a report
loadpath = "example/sparse"
fname = "0013"
d, d_test = generate_d()
ck = ChemKinetics(funcptr_cr, d, d_test=d_test)
ck.load(loadpath, fname)
print(chem_equation_with_oxalate(ck.l, ck.r, k=ck.k, chemformula=ck.chemformula, conservation=ck.conservation))
savename = "example/report.pdf"
report(loadpath, fname, savename)

( 1) Mn+7 + Mn+2     -> Mn+3 + Mn+6     k = 48.67
( 2) Mn+7 + C2O4-2   -> Mn+6 + C2O4-1   k = 0.03
( 3) Mn+3 + C2O4-2   -> Mn+2 + C2O4-1   k = 10.85
( 4) Mn+2 + Mn+6     -> 2 Mn+4          k = 9996.27
( 5) Mn+2 + Mn+4     -> 2 Mn+3          k = 9999.74
( 6) Mn+2 + C2O4-1   -> Mn+3 + C2O4-2   k = 54.30
( 7) C2O4-2 + Mn+4   -> Mn+3 + C2O4-1   k = 35.66
( 8) Mn+7 + C2O4-1   -> 2 CO2 + Mn+6    k = 1049.52
( 9) Mn+3 + C2O4-1   -> Mn+2 + 2 CO2    k = 385.73
(10) C2O4-2 + Mn+6   -> 2 CO2 + Mn+4    k = 478.75
(11) Mn+4 + C2O4-1   -> Mn+3 + 2 CO2    k = 9731.23

